# GPT

## 构造数据集
1. itos stoi字典映射
2. decoder encoder(又很多种方法，例如谷歌的SentencePiece就是一种sub_word unit,GPT有一个TikToken的库，使用字节对编码  ),下来可以复现一下
3. data是encoder 后的数据，是一个很长的张量，接下来分训练集和测试集，每次训练有一定的长度和最大长度(变量名为block_size )
4. 构造context和target,让gpt能够适应各种上下文
5. 接下来纪念性并行操作
6. embedding

## 简单的前向传播和采样
1. 注意交叉熵的维度，要调整一下
2. 注意这里虽然从一个开始生成，不断拼接，但是仍然预测时仅仅考虑了前一个词，没有考虑历史信息，karpathy这样做的目的是方便以后改良，加入历史信息

接下来他转移阵地到vscode,新添加了cuda

## 构建注意力
讲到第三种方法时，weight从零开始初始化，可以看作是交互强度或亲和力，告诉我们希望从过去每个标记中聚合或平均多少内容，mask分界线表示未来和过去的标记不能交流，把他们设置成负无穷大，我们表示不会从那些标记中聚合。所以这些会经过softmax处理，通过矩阵乘法进行聚合。
虽然初始化设置为零，但标记间的亲和力不会一直保持为零，他们将会是数据依赖的

### 位置编码
不仅要encode词元的身份，还要编码他们的位置

### 改进之前的注意力
1. 之前仅仅做了平均处理，这样的注意力是平滑的
2. 现在每个节点或每个位置的每个词元都会发出两个向量，query(what am i looking for) ,key(what do i contain)
3. V is the thing that gets aggregated for the purposes of this single head between the different nides  


1. 注意力机制是一种通信机制，可以想象一个有向图，有很多节点，基本上节点之间有边连接，每个节点有一些信息向量，它通过指向它的所有节点中进行加权和来聚合信息，这是以数据依赖的方式进行的，因此取决于任何时间点储存
而我们构建的字注意力机制有点不同， 假设有8个节点，第n个节点由前面n-1个节点指向，这就是有向图在自回归里的应用，但原则上，注意力可以应用于任何任意的带向图，只是节点之间的通信机制
2. 没有空间的概念，注意力只是在这个图上作用于的一组向量，这些节点默认不知道它们在空间中的位置，所以我们要对他们进行空间编码，给他们铆定到特定位置的信息，一遍它们大致知道自己的位置，这也与卷积不同，在某个输入进行卷积操作，信息在空间中有一个非常特定的布局，filter在空间中移动起作用。而注意力仅仅是一组向量
3. 批次(batch)维度上的元素是独立的，跨batch的元素永远不会互相通信，总是被独立处理。即4组独立的8个向量同时被处理，但不会跨组运算（从矩阵乘法理解一下）
4. 有事我们也希望元素间都可以通信（不屏蔽未来信息），encoder模块，会删除掩码的那行代码。而解码器，具有自回归格式，需要用三角矩阵进行掩码，屏蔽未来信息
5. cross attention.自注意里的原因是qurry,key,value来自同一个源。但在一下encoder-decoder的teansformer中，qurry可以由X产生，但键和值来自一个完全独立的外部源，有时来自编码了一些我们希望依赖的上下文的编码器块

论文中还除以了头的平方根
因为这是为了控制方差为一，因为结果要经过softmax向量的处理，应该理想初始化分散一点，方差过大有可能会初始化类似one-hot向量，softmax过于尖锐，基本上只能从单个节点聚合信息，这样每一个节点只能从另一个单个点节点聚合信息，这不是我们想要的，尤其时在初始化阶段


过了多头注意力
然后是前馈嵌入（在注意力后，独立的处理信息）


接着在通信和计算之间穿插进行,封装进block，然后又多个block 

## 残差连接
在block 里面添加残差连接


## layernorm
底层原理仅仅把BN计算的维度从0改成1，然后删除缓存，动量

## pre-norm Formulation(与原始论文不同之处)


原始论文 --- post-norm
x = x + Dropout(Attention(...))
x = LayerNorm(x)               # <- LayerNorm 在残差之后（Post-Norm）

x = x + Dropout(FeedForward(...))
x = LayerNorm(x)               # <- 又一次 Post-Norm

现在主流框架 ---Pre-Norm(接在残差连接前)
attn_out = Attention(LayerNorm(x))
x = x + Dropout(attn_out)      # <- Pre-Norm
   

ff_out = FeedForward(LayerNorm(x))
x = x + Dropout(ff_out)        # <- Pre-Norm again


## 接着再封装，将block的层数作为参数

## dropout
在前馈网络最后（残差网络前），多头注意力最后一层

## 总结
这节课我们仅仅完成了解码器部分的实现，没有编码器和交叉注意力。因为本次的任务是仅仅生成文本。而原始论文中是机器翻译任务，有encoder架构，没有三角形掩码，允许自由交流。

于是有了交叉注意力，qurry仍然由x产生，key和value变成了由旁边encoder尾部节点输出产生

## 引入nanoGPT(关注于预训练步骤)
里面的train.py标复杂，有更多选项，比如gpu,并行式等
model.py比较熟悉，有因果注意力模块。本节中是构建单个head类，再构建多头注意力，明确的连接起来(concat)
在model.py里,所有这些都在单个因果注意力内部以batch的形式，于是又了四个维度。在数学上等价，但更高效。
除此之外，用gelu非线性激活函数而不是relu(因为openai这样用)

## 训练chatgpt的步骤
1. 预训练:在互联网上大量训练，仅仅尝试让一个只包含解码器的transformer生成文本。openai与我们目前不同对是他们用的是subword chunks of words大约有50000个词汇元素
他们的序列更紧凑。他们的参数相当多，需要成千上万的GPU相互通信。不过仅仅依靠与训练的结果往往答非所问
2. 微调(从补全器变成问答器)
- 根据问题-答案微调
- 奖励模型
- 运行多轮PPO,一种策略梯度（近端策略优化）